### Test case 63: Figure out why results in line 3027 output (33554689) and expected (33554688) are different.

Thanks to Leetcoder for helping me figure this out:
https://youtu.be/jUVnLi4uGzE

In [2]:
!diff output.txt expected.txt

3027c3027
< 33554689
---
> 33554688


In [10]:
print(bin(33554689))
print(bin(33554688))

0b10000000000000000100000001
0b10000000000000000100000000


In [5]:
!sed "3027q;d" output.txt

33554689


In [7]:
!sed "3027q;d" expected.txt

33554688


In [12]:
!sed "3027q;d" input_queries.txt

33554432,256


In [20]:
with open('input_queries.txt', 'r') as queries_f:
    queries = queries_f.readlines()
    print(queries[3027 - 1])
    print(bin(int(queries[3027 - 1].split(',')[0])))

33554432,256

0b10000000000000000000000000


In [21]:
nums_f = open('input_nums.txt')
nums = [int(line) for line in nums_f]
nums_f.close()

queries_f = open('input_queries.txt', 'r')
queries = [[int(line.split(',')[0]), int(line.split(',')[1])] for line in queries_f]
queries_f.close()

In [30]:
x, m = queries[3027 - 1]
print(f'x = {x} ({bin(x)}), m = {m} ({bin(m)})')
num = max((n for n in nums if n <= m), key=lambda n: n^x, default=-1) # from https://stackoverflow.com/a/66083397/8694392
print('num of nums = ', num, ' = ', bin(num))
print(f'x ^ num = {x ^ num} = {bin(x ^ num)}')


x = 33554432 (0b10000000000000000000000000), m = 256 (0b100000000)
num of nums =  256  =  0b100000000
x ^ num = 33554688 = 0b10000000000000000100000000


Modifying my code to print chosen num when x = 33554432 and m = 256.
```cpp
if (x == 33554432 && m == 256) {
    cout << "Special case x = 33554432, m = 256" << endl << "\tchosen num = " << current_amount << ", xor: " << int(x ^ current_amount) << endl;
}
```
Output:
```
Special case x = 33554432, m = 256
	chosen num = 257, xor: 33554689
```
For my code in this case, num = 257????

# TODO: Figure out why 257 is chosen, if m == 256 and num should be chosen so num <= m

In [32]:
print(f'chosen num = 257 ({bin(257)}), xor output = 33554689 = {bin(33554689)}')

chosen num = 257 (0b100000001), xor output = 33554689 = 0b10000000000000000100000001



```
x        = 0b 0000 0010  0000 0000  0000 0000  0000 0000
m        = 0b 0000 0000  0000 0000  0000 0001  0000 0000
out num  = 0b 0000 0000  0000 0000  0000 0001  0000 0001
num      = 0b 0000 0000  0000 0000  0000 0001  0000 0000
output   = 0b 0000 0010  0000 0000  0000 0001  0000 0001
expected = 0b 0000 0010  0000 0000  0000 0001  0000 0000
```

# Update: Solution Found!!!!
I think the problem was most likely when building the trie in `addToBinaryTrie(int val)` that:
The min value in the lowermost level "leaf" nodes of the trie (corresponding to the rightmost least significant bit) was not set correctly. So at these nodes the min value of the trie could still be set to INT_MAX which could cause the algorithm to consider both nodes to have a min value of INT_MIN instead of the value of the node as it shouuld be. This could potentially causing the algorithm to not take the min into account when choosing between the 2 leaf branches, and so to choose the final node for the rightmost least significant bit based on the least significant bit in x instead of based on which branch has a min <= m.

The trie should have 33 levels (1 root + 32 levels for each bit in C++ int). So the `for` loop setting the min value was setting the min before traversing to the next node for the next bit, and then stopping after getting to the next bit. So I think I fixed this by first setting the min of the root node manually, and then in the 'for' loop traverse to the next lower level before setting the min there.


### The fix:

```cpp
void addToBinaryTrie(int val) {
    ...
    if (root == nullptr) {
        root = new Node();
    }
    Node* current = root;
    if (root->min > val) { // Set the root min separately.
        root->min = val;
    }
    for(int i = 0; i < sizeof(int) * 8; i++) { // Runs for 32 iterations.

        ... add bits from val into trie ...
        
        if (current->min > val) { // Set the min before next iteration.
            current->min = val;
        }
    }
    // Trie has 32 + 1 = 33 levels, and the min should probably be set correctly for nodes in each level.
}
```



#### Different trie approach from a Stackoverflow responder

In [33]:
# From https://stackoverflow.com/a/66083223/8694392
from typing import List


class Trie:
    def __init__(self):
        self.root = {}
    
    def add(self, n):
        p = self.root
        for bitpos in range(31, -1, -1):
            bit = (n >> bitpos) & 1
            if bit not in p:
                p[bit] = {}
            p = p[bit]
    
    def query(self, n):
        p = self.root
        ret = 0
        if not p:
            return -1
        for bitpos in range(31, -1, -1):
            bit = (n >> bitpos) & 1
            inverse = 1 - bit
            if inverse in p:
                p = p[inverse]
                ret |= (1 << bitpos)
            else:
                p = p[bit]
                
        return ret

class Solution:
    def maximizeXor(self, nums: List[int], queries: List[List[int]]) -> List[int]:
        
        n = len(nums)
        trie = Trie()
        q = sorted(enumerate(queries), key = lambda x: x[1][1])
        nums.sort()
        res = [-1] * len(queries)
        i = 0
        for index, (x, m) in q:
            while i < n and nums[i] <= m:
                trie.add(nums[i])
                i += 1
            res[index] = trie.query(x)
        return res

In [34]:
output = Solution().maximizeXor(nums, queries)
expected_f = open('expected.txt', 'r')
expected = [int(line) for line in expected_f]
expected_f.close()